# Feature documentation

## Features Actuales

 - factores c/v c/s
 - diferencia entre primer y ultimo dia que se metio en el mes 5
 - primer dia del mes 5 que se metio, ultimo dia del mes 5 y que numero de dia de la semana fue
 - device type que tipo fue, y cuantas veces se metio en ese
 - new or returning y cuantas veces lo fueron
 - cuantos meses distintos se metieron
 - dias distintos que vio un modelo 
 - checkout y visited site, promedio, suma, std de todos los dias
    
## Features probados que no funcionaron y otras cosas que tampoco:

   ### Firs
  - Numero de veces que realizo cada evento en dataframe
   ### Algoritmo 0.63 con RandomFores
  - Use random forest para ver la importancia de lo features de las busquedas y de los modelos y me quede con los mejores 10, el resultado fue menor con 0.61
   
## Best Score: 0. 
  - commit 0.71 
  - Max, min date.
               
## Cosas a probar:
 
 

In [1]:
import sys #access to system parameters https://docs.python.org/3/library/sys.html

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features

import matplotlib #collection of functions for scientific and publication-ready visualization

import numpy as np #foundational package for scientific computing

import scipy as sp #collection of functions for scientific computing and advance mathematics

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook

import sklearn #collection of machine learning algorithms

#misc libraries
import random
import time
import datetime as dt

#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

import featuretools as ft
from sklearn.feature_extraction.text import CountVectorizer

-------------------------


In [2]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
import xgboost as xgb

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn.model_selection import train_test_split

from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8



In [3]:
data_raw = pd.read_csv('events_up_to_01062018.csv')
data_val = pd.read_csv('labels_training_set.csv')

In [4]:
data_raw.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_labels = data_val.copy(deep=True) 
df = data_raw.copy(deep=True)

In [6]:
pd.set_option('display.max_columns', 23)

In [7]:
df.describe(include= 'all')

,timestamp,event,person,url,sku,model,condition,storage,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
count,2341681,2341681,2341681,191131,1.320530e+06,1321513,1320530,1320530,1320530,505949,113763,11201,191286,106406,204069,204069,204069,204069,204069,204069,204066,204069,204069
unique,1490912,11,38829,248,NaN,208,5,8,63,52267,10964,14,23,4,7,2,2206,122,51,4,393,131,366
top,2018-05-31 01:59:16,viewed product,c76b8417,/,NaN,iPhone 6,Bom,16GB,Preto,"2820,6706,6720,2750,6649,7251,6663,12604,7224,...",Iphone,CustomerService,google,Google,Paid,Returning,Unknown,Sao Paulo,Brazil,Smartphone,360x640,Windows 7,Chrome 66.0
freq,14,1248124,4438,64187,NaN,107262,547617,442096,314925,2606,2577,5239,123354,105195,91753,165827,36866,57304,197699,103502,73234,46648,57953
mean,NaN,NaN,NaN,NaN,6.899178e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,4.028042e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,7.100000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,2.929000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,7.057000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,1.001400e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
persons = (df.drop_duplicates('person'))['person'].to_frame()
persons_to_train = df_labels['person'].to_frame()
persons_to_predict = persons.loc[~persons['person'].isin(persons_to_train['person'])]


print(persons_to_train.shape)
print(persons_to_predict.shape)


(19414, 1)
(19415, 1)


In [9]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values(['timestamp','event'])
df = df.reset_index(drop=True)



In [10]:
df_dates_per_month = persons
df_months = df
df_months['month'] = df_months['timestamp'].dt.month
df_months['days'] = df_months['timestamp'].dt.day

In [11]:
#for x in range(1,6):
#    df_month = df_months.loc[df['month'] == x]
 #   df_dates = df_month.groupby('person').agg({'timestamp':['max', 'min']}).reset_index()
  #  df_dates.columns = ['person', 'last month '+str(x), 'first month '+ str(x)]
   # df_dates['diferencia'+str(x)] = (df_dates['last month '+str(x)] - df_dates['first month '+ str(x)]).dt.days
    #print(df_dates.head(1))
     #df_dates_per_month = df_dates_per_month.merge(df_dates, on = 'person', how='left')
    

In [12]:
df_month = df_months.loc[df['month'] == 5]
df_dates = df_month.groupby('person').agg({'timestamp':['max', 'min']}).reset_index()
df_dates.columns = ['person', 'last month 5', 'first month 5']
df_dates['diferencia 5'] = (df_dates['last month 5'] - df_dates['first month 5']).dt.days



In [13]:
df_dates_per_month =df_dates_per_month.merge(df_dates, on='person', how='left')

In [14]:
es = ft.EntitySet(id = 'person')
es = es.entity_from_dataframe(entity_id = 'person_id', dataframe = df_dates_per_month, index = 'person')

In [15]:
%time feat_date, feature_names = ft.dfs(entityset=es, target_entity='person_id', max_depth = 2)

CPU times: user 1.18 s, sys: 0 ns, total: 1.18 s
Wall time: 1.17 s


In [16]:
feat_date.head()

,diferencia 5,DAY(last month 5),DAY(first month 5),YEAR(last month 5),YEAR(first month 5),MONTH(last month 5),MONTH(first month 5),WEEKDAY(last month 5),WEEKDAY(first month 5)
person,,,,,,,,,
0008ed71,0.0,17.0,17.0,2018.0,2018.0,5.0,5.0,3.0,3.0
00091926,27.0,31.0,3.0,2018.0,2018.0,5.0,5.0,3.0,3.0
00091a7a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000ba417,9.0,26.0,17.0,2018.0,2018.0,5.0,5.0,5.0,3.0
000c79fe,0.0,29.0,29.0,2018.0,2018.0,5.0,5.0,1.0,1.0


In [17]:
feat_date['person'] = feat_date.index
feat_date = feat_date.reset_index(drop=True)
feat_date.head()

,diferencia 5,DAY(last month 5),DAY(first month 5),YEAR(last month 5),YEAR(first month 5),MONTH(last month 5),MONTH(first month 5),WEEKDAY(last month 5),WEEKDAY(first month 5),person
0,0.0,17.0,17.0,2018.0,2018.0,5.0,5.0,3.0,3.0,0008ed71
1,27.0,31.0,3.0,2018.0,2018.0,5.0,5.0,3.0,3.0,00091926
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00091a7a
3,9.0,26.0,17.0,2018.0,2018.0,5.0,5.0,5.0,3.0,000ba417
4,0.0,29.0,29.0,2018.0,2018.0,5.0,5.0,1.0,1.0,000c79fe


In [18]:
feat_date.describe(include = 'all')



,diferencia 5,DAY(last month 5),DAY(first month 5),YEAR(last month 5),YEAR(first month 5),MONTH(last month 5),MONTH(first month 5),WEEKDAY(last month 5),WEEKDAY(first month 5),person
count,37143.000000,37143.000000,37143.000000,37143.0,37143.0,37143.0,37143.0,37143.000000,37143.000000,38829
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38829
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ff89d18f
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
mean,4.732763,24.545379,19.564306,2018.0,2018.0,5.0,5.0,2.551813,2.571359,NaN
std,7.612190,5.688888,8.168224,0.0,0.0,0.0,0.0,1.695478,1.799570,NaN
min,0.000000,1.000000,1.000000,2018.0,2018.0,5.0,5.0,0.000000,0.000000,NaN
25%,0.000000,21.000000,15.000000,2018.0,2018.0,5.0,5.0,1.000000,1.000000,NaN
50%,0.000000,25.000000,21.000000,2018.0,2018.0,5.0,5.0,2.000000,2.000000,NaN
75%,7.000000,30.000000,26.000000,2018.0,2018.0,5.0,5.0,3.000000,4.000000,NaN


In [19]:
feat_date['diferencia 5'].fillna(int(feat_date['diferencia 5'].median()), inplace = True)
feat_date['DAY(last month 5)'].fillna(int(feat_date['DAY(last month 5)'].mean()), inplace = True)
feat_date['DAY(first month 5)'].fillna(int(feat_date['DAY(first month 5)'].mean()), inplace = True)
feat_date['WEEKDAY(last month 5)'].fillna(int(feat_date['WEEKDAY(last month 5)'].mean()), inplace = True)
feat_date['WEEKDAY(first month 5)'].fillna(int(feat_date['WEEKDAY(first month 5)'].mean()), inplace = True)
lista = ['diferencia 5','DAY(last month 5)', 'DAY(first month 5)','WEEKDAY(last month 5)','WEEKDAY(first month 5)','person']

feat_date = feat_date[lista]

 


In [20]:
feat_date = persons.merge(feat_date, on='person', how='left')

# Feature Cantidad de dias que vio un mismo modelo

#### Dias que vio su producto mas visto

In [21]:
df['day'] = df['timestamp'].dt.day

In [22]:
df_model_viewed = df.loc[df['event'] == 'viewed product']
df_model_viewed = df_model_viewed.loc[df_model_viewed['month'] == 5]

df_model_viewed['count']= 1

In [23]:
df_model_viewed = df_model_viewed.groupby(['person','month','day', 'model']).agg({'count':'sum'}).reset_index()
df_model_viewed.head()

,person,month,day,model,count
0,00091926,5,3,Motorola Moto X Style,2
1,00091926,5,3,Samsung Galaxy A7 2017,1
2,00091926,5,3,iPhone 6S,1
3,00091926,5,3,iPhone 7,2
4,00091926,5,5,Motorola Moto X Style,2


In [24]:
df_model_viewed['days'] = 1
df_model_viewed = df_model_viewed.groupby(['person', 'model']).agg({'count':'sum','days':'sum'}).reset_index()


In [25]:
df_model_viewed = df_model_viewed.sort_values(by=['count'],ascending = [False]).drop_duplicates(subset='person',keep='first')

In [26]:
df_model_viewed = df_model_viewed.drop(['model','count'],axis=1)

In [27]:
df_model_viewed = df_model_viewed.merge(persons, on='person', how = 'right')

In [28]:
df_model_viewed.head()

,person,days
0,aa297476,16.0
1,b793fc69,11.0
2,069c5926,11.0
3,eb6c498c,22.0
4,455b577c,15.0


## Mean


In [29]:
df_model_viewed_mean = df.loc[df['event'] == 'viewed product']
df_model_viewed_mean = df_model_viewed_mean.loc[df_model_viewed_mean['month'] == 5]

df_model_viewed_mean['count']= 1

In [30]:
df_model_viewed_mean = df_model_viewed_mean.groupby(['person','month','day', 'model']).agg({'count':'sum'}).reset_index()
df_model_viewed_mean.head()

,person,month,day,model,count
0,00091926,5,3,Motorola Moto X Style,2
1,00091926,5,3,Samsung Galaxy A7 2017,1
2,00091926,5,3,iPhone 6S,1
3,00091926,5,3,iPhone 7,2
4,00091926,5,5,Motorola Moto X Style,2


In [31]:
df_model_viewed_mean['days'] = 1
df_model_viewed_mean = df_model_viewed_mean.groupby(['person', 'model']).agg({'count':'sum','days':'sum'}).reset_index()
df_model_viewed_mean.head()

,person,model,count,days
0,00091926,LG G4 H815P,1,1
1,00091926,LG G5 SE,1,1
2,00091926,Lenovo Vibe K5,1,1
3,00091926,Motorola Moto G4 Play DTV,5,1
4,00091926,Motorola Moto G5,2,1


In [32]:
df_model_viewed_mean = df_model_viewed_mean.groupby(['person']).agg({'count':['mean'],'days':['mean','max']}).reset_index()
df_model_viewed_mean.head()

person      count      days    
                  mean      mean max
0  00091926  10.333333  2.500000  16
1  000ba417   5.884615  1.346154   3
2  000c79fe   3.000000  1.000000   1
3  000e4d9e   9.162162  1.837838   9
4  000e619d   1.217391  1.043478   2

In [33]:
df_model_viewed_mean.columns = ['person','viewed_model_mean','models_viewed_days_mean','models_viewed_days_max']


In [34]:
df_model_viewed_mean.head()

,person,viewed_model_mean,models_viewed_days_mean,models_viewed_days_max
0,00091926,10.333333,2.500000,16
1,000ba417,5.884615,1.346154,3
2,000c79fe,3.000000,1.000000,1
3,000e4d9e,9.162162,1.837838,9
4,000e619d,1.217391,1.043478,2


In [35]:
feat_day_model_stats = df_model_viewed
feat_day_model_stats = persons.merge(feat_day_model_stats,on='person', how='left')


In [36]:
#feat_day_model_stats = feat_day_model_stats.merge(df_model_viewed_mean,on='person', how='left')
feat_day_model_stats.head()

,person,days
0,4886f805,1.0
1,ad93850f,3.0
2,0297fc1e,14.0
3,2d681dd8,1.0
4,cccea85e,10.0


## XGboost entrenamiento para evaluar feature solo

In [37]:
df_train = df_labels.merge(df_model_viewed_mean , left_on='person', right_on='person' , how='inner')

In [38]:
df_train.head()

,person,label,viewed_model_mean,models_viewed_days_mean,models_viewed_days_max
0,0566e9c1,0,1.916667,1.083333,2
1,abe7a2fb,0,4.666667,1.333333,2
2,34728364,0,2.181818,1.181818,2
3,87ed62de,0,9.000000,2.000000,2
4,cde431db,0,4.000000,1.500000,2


In [39]:
X, y = df_train.iloc[:,2:],df_train.iloc[:,1]
X.head()

,viewed_model_mean,models_viewed_days_mean,models_viewed_days_max
0,1.916667,1.083333,2
1,4.666667,1.333333,2
2,2.181818,1.181818,2
3,9.000000,2.000000,2
4,4.000000,1.500000,2


In [40]:
import xgboost as xgb
model = xgb.XGBClassifier(objective ='reg:linear', 
                colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 6,
                subsample = 0.8,
                gamma = 1,
                n_estimators = 10)



Este es el arbol con sus hiperparametros

In [41]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [42]:
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [43]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,  model.predict_proba(X_test)[:,1])

0.5839875519661402

# Campaign ppl


In [44]:
(df.loc[df['event'] == 'ad campaign hit']).drop_duplicates('person').shape

(31900, 26)

In [45]:
df_campaign = (df.loc[df['event'] == 'ad campaign hit'])
df_campaign['count'] = 1

In [46]:
df_campaign = df_campaign.groupby(['person','month', 'day']).agg({'count':'sum'}).reset_index()


In [47]:
df_campaign = df_campaign.groupby('person').agg({'count':['sum' , 'std','mean']}).reset_index()

In [48]:
df_campaign.head()

person count                    
              sum       std      mean
0  00091926    15  0.504525  1.363636
1  00091a7a     1       NaN  1.000000
2  000ba417     1       NaN  1.000000
3  000c79fe     1       NaN  1.000000
4  000e4d9e    19  8.386497  6.333333

# Visited ppl


In [49]:
df['event'].value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

In [50]:
print((df.loc[df['event'] == 'viewed product']).drop_duplicates('person').shape)

print((df.loc[df['event'] == 'visited site']).drop_duplicates('person').shape)

print((df.loc[df['event'] == 'ad campaign hit']).drop_duplicates('person').shape)

print((df.loc[df['event'] == 'checkout']).drop_duplicates('person').shape)



(37130, 26)
(38242, 26)
(31900, 26)
(32833, 26)


In [51]:
(df.loc[df['event'] == 'visited site']).drop_duplicates('person').shape

(38242, 26)

In [52]:
df_visited = (df.loc[df['event'] == 'visited site'])
df_visited = df_visited.loc[df_visited['month']==5]

df_visited['count'] = 1

In [53]:
df_visited = df_visited.groupby(['person','month', 'day']).agg({'count':'sum'}).reset_index()


In [54]:
df_visited = df_visited.groupby('person').agg({'count':['sum' , 'std','mean']}).reset_index()

In [55]:
df_visited.head()

person count                    
              sum       std      mean
0  0008ed71     2       NaN  2.000000
1  00091926    34  1.100964  1.545455
2  000ba417     6  1.000000  2.000000
3  000c79fe     1       NaN  1.000000
4  000e4d9e    13  0.726483  1.444444

In [56]:
df_visited.columns = ['person' , 'visited_sum', 'visited_std', 'visited_mean']

# Checkout ppl

In [57]:
(df.loc[df['event'] == 'checkout']).drop_duplicates('person').shape

(32833, 26)

In [58]:
df_checkout = (df.loc[df['event'] == 'checkout'])
df_checkout = df_checkout.loc[df_checkout['month']==5]
df_checkout['count'] = 1

In [59]:
df_checkout = df_checkout.groupby(['person','month', 'day']).agg({'count':'sum'}).reset_index()


In [60]:
df_checkout = df_checkout.groupby('person').agg({'count':['sum' ,'mean']}).reset_index()

In [61]:
df_checkout.head()

person count     
              sum mean
0  0008ed71     3  3.0
1  00091926     2  1.0
2  000ba417     6  2.0
3  000c79fe     1  1.0
4  000e4d9e     1  1.0

In [62]:
df_checkout = persons.merge(df_checkout, on='person', how='left')

In [63]:
df_checkout.shape

(38829, 3)

In [64]:
df_checkout.columns = ['person' , 'checkout_sum', 'checkout_mean']

# Checkout ppl

In [65]:
(df.loc[df['event'] == 'checkout']).drop_duplicates('person').shape

(32833, 26)

In [66]:
df['week']=df['timestamp'].dt.week

In [67]:
df_checkout2 = (df.loc[df['event'] == 'checkout'])
df_checkout2 = df_checkout2.loc[df_checkout2['month']==5]
df_checkout2['count'] = 1

In [68]:
df_checkout2 = df_checkout2.groupby(['person','month', 'week']).agg({'count':'sum'}).reset_index()


In [69]:
df_checkout2 = df_checkout2.groupby('person').agg({'count':['sum' ,'mean']}).reset_index()

In [70]:
df_checkout2.head()

person count     
              sum mean
0  0008ed71     3  3.0
1  00091926     2  1.0
2  000ba417     6  3.0
3  000c79fe     1  1.0
4  000e4d9e     1  1.0

In [71]:
df_checkout2 = persons.merge(df_checkout2, on='person', how='left')

In [72]:
df_checkout2.shape

(38829, 3)

In [73]:
df_checkout2.columns = ['person' , 'checkout_sum', 'checkout_mean']

# Viewed ppl

In [74]:
(df.loc[df['event'] == 'viewed product']).drop_duplicates('person').shape

(37130, 27)

In [75]:
df_viewed = (df.loc[df['event'] == 'viewed product'])
df_viewed['count'] = 1

In [76]:
df_viewed = df_viewed.groupby(['person','month', 'day']).agg({'count':'sum'}).reset_index()


In [77]:
df_viewed = df_viewed.groupby('person').agg({'count':['sum' , 'std','mean']}).reset_index()

In [78]:
df_viewed.head()

person count                      
              sum        std       mean
0  00091926   372  18.899735  18.600000
1  00091a7a     3        NaN   3.000000
2  000ba417   153  16.093477  51.000000
3  000c79fe     3        NaN   3.000000
4  000e4d9e   339  49.401417  37.666667

In [79]:
df_viewed = persons.merge(df_viewed, on='person', how='left')

In [80]:
df_viewed.shape

(38829, 4)

# Conversion ppl

In [81]:
(df.loc[df['event'] == 'conversion']).drop_duplicates('person').shape

(4293, 27)

In [82]:
df_conversion = (df.loc[df['event'] == 'conversion'])
df_conversion = df_conversion.loc[df_conversion['month']==5]
df_conversion = df_conversion.loc[df_conversion['day']>15]

df_conversion['count'] = 1

In [83]:
df_conversion = df_conversion.groupby(['person','month', 'day']).agg({'count':'sum'}).reset_index()


In [84]:
df_conversion = df_conversion.groupby('person').agg({'count':['sum','mean']}).reset_index()

In [85]:
df_conversion.head()

person count     
              sum mean
0  000ba417     1  1.0
1  001001be     1  1.0
2  001804a2     1  1.0
3  0019e639     1  1.0
4  001b0bf9     1  1.0

In [86]:
df_conversion = persons.merge(df_conversion, on='person', how='left')

In [87]:
df_conversion = df_conversion.fillna(0)
df_checkout2.columns = ['person' , 'conversion_sum', 'conversion_mean']
df_conversion.shape

(38829, 3)

- En otros notebooks hice un analisis mas exhaustivo de cuales transformaciones generaban los features mas optimos

- Los mas optimos fueron __checkout__ y __visited site__

In [88]:
feat_checkout = df_checkout
feat_checkout2 = df_checkout2

feat_visited = df_visited

# Junto mis features

In [107]:
feat_diff = pd.read_csv('feat_dummies.csv')
feat_diff.head()

,person,Dourado,Preto,Bom,Muito Bom,Direct,Email,Organic,Paid,Referral,Social,Computer,Smartphone,Tablet
0,4886f805,5.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,ad93850f,5.0,0.0,2.0,7.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,5.0,0.0
2,0297fc1e,31.0,0.0,73.0,48.0,17.0,0.0,0.0,4.0,8.0,0.0,0.0,29.0,0.0
3,2d681dd8,1.0,5.0,14.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0
4,cccea85e,32.0,204.0,200.0,347.0,1.0,0.0,14.0,6.0,1.0,0.0,22.0,0.0,0.0


In [90]:
feat_082= pd.read_csv('feat_082.csv')
feat_082.head()

,person,diferencia 5,DAY(last month 5),DAY(first month 5),WEEKDAY(last month 5),WEEKDAY(first month 5)
0,4886f805,0.0,18.0,18.0,4.0,4.0
1,ad93850f,0.0,14.0,14.0,0.0,0.0
2,0297fc1e,12.0,22.0,10.0,1.0,3.0
3,2d681dd8,0.0,27.0,27.0,6.0,6.0
4,cccea85e,0.0,11.0,11.0,4.0,4.0


In [108]:
features = persons
features = features.merge(feat_checkout, on='person', how='left')
features = features.merge(feat_checkout2, on='person', how='left')
features = features.merge(df_conversion, on='person', how='left')
features = features.merge(feat_diff, on='person', how='left')
features = features.merge(df_viewed, on='person', how='left')
features = features.merge(feat_082, on='person', how='left')

#features = features.merge(feat_diff, on='person', how='left')
#features = features.merge(features_rance, on='person', how='left')

features = features.merge(feat_date, on='person', how='left')


In [109]:
features.columns

Index([                  'person',             'checkout_sum',
                  'checkout_mean',           'conversion_sum',
                'conversion_mean',       '('count', 'sum')_x',
            '('count', 'mean')_x',                  'Dourado',
                          'Preto',                      'Bom',
                      'Muito Bom',                   'Direct',
                          'Email',                  'Organic',
                           'Paid',                 'Referral',
                         'Social',                 'Computer',
                     'Smartphone',                   'Tablet',
             '('count', 'sum')_y',           ('count', 'std'),
            '('count', 'mean')_y',           'diferencia 5_x',
            'DAY(last month 5)_x',     'DAY(first month 5)_x',
        'WEEKDAY(last month 5)_x', 'WEEKDAY(first month 5)_x',
                 'diferencia 5_y',      'DAY(last month 5)_y',
           'DAY(first month 5)_y',  'WEEKDAY(last month

In [110]:
features.head()

,person,checkout_sum,checkout_mean,conversion_sum,conversion_mean,"('count', 'sum')_x","('count', 'mean')_x",Dourado,Preto,Bom,Muito Bom,...,"('count', 'mean')_y",diferencia 5_x,DAY(last month 5)_x,DAY(first month 5)_x,WEEKDAY(last month 5)_x,WEEKDAY(first month 5)_x,diferencia 5_y,DAY(last month 5)_y,DAY(first month 5)_y,WEEKDAY(last month 5)_y,WEEKDAY(first month 5)_y
0,4886f805,1.0,1.0,1.0,1.0,0.0,0.0,5.0,0.0,0.0,1.0,...,4.000000,0.0,18.0,18.0,4.0,4.0,0.0,18.0,18.0,4.0,4.0
1,ad93850f,1.0,1.0,1.0,1.0,0.0,0.0,5.0,0.0,2.0,7.0,...,6.666667,0.0,14.0,14.0,0.0,0.0,7.0,22.0,14.0,1.0,0.0
2,0297fc1e,2.0,1.0,2.0,1.0,0.0,0.0,31.0,0.0,73.0,48.0,...,7.214286,12.0,22.0,10.0,1.0,3.0,26.0,28.0,2.0,0.0,2.0
3,2d681dd8,1.0,1.0,1.0,1.0,0.0,0.0,1.0,5.0,14.0,0.0,...,6.500000,0.0,27.0,27.0,6.0,6.0,9.0,27.0,18.0,6.0,4.0
4,cccea85e,1.0,1.0,1.0,1.0,0.0,0.0,32.0,204.0,200.0,347.0,...,56.846154,0.0,11.0,11.0,4.0,4.0,23.0,31.0,7.0,3.0,0.0


## XGboost entrenamiento

In [111]:
df_train = df_labels.merge(features , left_on='person', right_on='person' , how='inner')

In [112]:
df_train.shape

(19414, 34)

In [113]:
X, y = df_train.iloc[:,2:],df_train.iloc[:,1]
X.head()

,checkout_sum,checkout_mean,conversion_sum,conversion_mean,"('count', 'sum')_x","('count', 'mean')_x",Dourado,Preto,Bom,Muito Bom,Direct,...,"('count', 'mean')_y",diferencia 5_x,DAY(last month 5)_x,DAY(first month 5)_x,WEEKDAY(last month 5)_x,WEEKDAY(first month 5)_x,diferencia 5_y,DAY(last month 5)_y,DAY(first month 5)_y,WEEKDAY(last month 5)_y,WEEKDAY(first month 5)_y
0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,7.0,16.0,6.0,8.0,...,11.500000,0.0,23.0,23.0,2.0,2.0,8.0,31.0,22.0,3.0,1.0
1,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,23.0,22.0,2.0,2.0,0.0,28.0,28.0,0.0,0.0
2,1.0,1.0,1.0,1.0,0.0,0.0,3.0,3.0,10.0,2.0,2.0,...,3.444444,0.0,29.0,29.0,1.0,1.0,27.0,29.0,2.0,1.0,2.0
3,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,8.000000,0.0,23.0,22.0,2.0,2.0,8.0,27.0,18.0,6.0,4.0
4,2.0,2.0,2.0,2.0,0.0,0.0,9.0,1.0,6.0,0.0,0.0,...,4.500000,0.0,18.0,18.0,4.0,4.0,0.0,19.0,18.0,5.0,4.0


## Xgboost

Para evaluar usen esta medida que me da valores muy parecidos a los de kaggle, para hacer las predicciones usen el otro

In [114]:
import xgboost as xgb
model = xgb.XGBClassifier(objective ='reg:linear', 
                colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 7,
                subsample = 0.9,
                gamma = 1,
                n_estimators = 65)



Este es el arbol con sus hiperparametros

In [115]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [116]:
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=65,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.9)

In [117]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,  model.predict_proba(X_test)[:,1])

0.8625838802448197

In [118]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [119]:


params = {"objective":"binary:logistic",'colsample_bytree':1,
          'learning_rate': 0.1, 'max_depth': 7, 'gamma': 1,'n_estimators': 65}
cv_val = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50, early_stopping_rounds=10,
                    metrics="auc", as_pandas=True, seed=123)

[13:58:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 20 pruned nodes, max_depth=7
[13:58:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 68 extra nodes, 10 pruned nodes, max_depth=7
[13:58:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 128 extra nodes, 10 pruned nodes, max_depth=7
[13:58:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 12 pruned nodes, max_depth=7
[13:58:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 14 pruned nodes, max_depth=7
[13:58:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 12 pruned nodes, max_depth=7
[13:58:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 70 extra nodes, 14 pruned nodes, max_depth=7
[13:58:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 78 extra nodes, 8 pruned nodes, max_depth=7
[13:58:

[13:58:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 136 extra nodes, 24 pruned nodes, max_depth=7
[13:58:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 16 pruned nodes, max_depth=7
[13:58:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 22 pruned nodes, max_depth=7
[13:58:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 128 extra nodes, 28 pruned nodes, max_depth=7
[13:58:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 12 pruned nodes, max_depth=7
[13:58:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 18 pruned nodes, max_depth=7
[13:58:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 20 pruned nodes, max_depth=7
[13:58:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 24 pruned nodes, max_depth=7


[13:58:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 132 extra nodes, 10 pruned nodes, max_depth=7
[13:58:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 84 extra nodes, 36 pruned nodes, max_depth=7
[13:58:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes, 22 pruned nodes, max_depth=7
[13:58:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 78 extra nodes, 20 pruned nodes, max_depth=7
[13:58:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 30 pruned nodes, max_depth=7
[13:58:49] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes, 12 pruned nodes, max_depth=7
[13:58:49] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 16 pruned nodes, max_depth=7
[13:58:49] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 100 extra nodes, 22 pruned nodes, max_depth=7
[13:58

In [120]:
cv_val

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.848646,0.003702,0.824264,0.003013
1,0.852128,0.003024,0.825214,0.002911
2,0.857599,0.002362,0.828670,0.003469
3,0.870417,0.007589,0.831725,0.006369
4,0.875543,0.009537,0.833762,0.007290
5,0.877708,0.008415,0.833015,0.007583
6,0.884162,0.007839,0.838964,0.002820
7,0.889379,0.006421,0.841549,0.004384
8,0.890163,0.006525,0.841811,0.005130
9,0.893360,0.007899,0.843580,0.004541


In [101]:
features.to_csv('features_con_cuidado.csv', index=False)

In [102]:
X_predict.head()

NameError: name 'X_predict' is not defined

In [106]:
X_predict = persons_to_predict.merge(features, on='person', how='left')
X_predict.head()


,person,checkout_sum,checkout_mean,conversion_sum,conversion_mean,"('count', 'sum')_x","('count', 'mean')_x","('count', 'sum')_y","(count, std)","('count', 'mean')_y",diferencia 5_x,DAY(last month 5)_x,DAY(first month 5)_x,WEEKDAY(last month 5)_x,WEEKDAY(first month 5)_x,diferencia 5_y,DAY(last month 5)_y,DAY(first month 5)_y,WEEKDAY(last month 5)_y,WEEKDAY(first month 5)_y
0,4886f805,1.0,1.0,1.0,1.0,0.0,0.0,4.0,NaN,4.000000,0.0,18.0,18.0,4.0,4.0,0.0,18.0,18.0,4.0,4.0
1,0297fc1e,2.0,1.0,2.0,1.0,0.0,0.0,404.0,5.228295,7.214286,12.0,22.0,10.0,1.0,3.0,26.0,28.0,2.0,0.0,2.0
2,2d681dd8,1.0,1.0,1.0,1.0,0.0,0.0,13.0,0.707107,6.500000,0.0,27.0,27.0,6.0,6.0,9.0,27.0,18.0,6.0,4.0
3,cccea85e,1.0,1.0,1.0,1.0,0.0,0.0,739.0,26.241971,56.846154,0.0,11.0,11.0,4.0,4.0,23.0,31.0,7.0,3.0,0.0
4,4c8a8b93,2.0,1.0,2.0,2.0,0.0,0.0,177.0,40.339807,35.400000,0.0,19.0,18.0,5.0,4.0,4.0,22.0,18.0,1.0,4.0


In [145]:
df_entrie = persons_to_predict.copy(deep=True)
df_entrie['label'] = model.predict_proba(X_predict.iloc[:,1:])[:,1]
df_entrie.head()

,person,label
0,4886f805,0.003992
2,0297fc1e,0.113473
3,2d681dd8,0.007088
4,cccea85e,0.099782
5,4c8a8b93,0.020375


In [146]:
df_entrie.shape

(19415, 2)

In [147]:
num = df_entrie._get_numeric_data()
num[num < 0] = 0

Obtengo un resultado con los que separe para el test mas arriba en :
```python
    X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)
    ```
###### Lo que hace es sacar las predicciones para X_test y evaluarlos con y_test
    

In [148]:
df_entrie['label'].nsmallest()

19941      0.000000
442203     0.000000
561785     0.000000
1459123    0.000000
541997     0.001166
Name: label, dtype: float32

In [149]:
df_entrie.to_csv('submit_kagggleeee.csv',index= False)

In [326]:
features.head()

,person,checkout_sum,checkout_mean,conversion_sum,conversion_mean,"('count', 'sum')_x","('count', 'mean')_x","(count_y, sum)","(count_y, std)","(count_y, mean)","(count, sum)","(count, std)","(count, mean)",diferencia 5,DAY(last month 5),DAY(first month 5),WEEKDAY(last month 5),WEEKDAY(first month 5)
0,4886f805,1.0,1.0,1.0,1.0,0.0,0.0,NaN,NaN,NaN,4.0,NaN,4.000000,0.0,18.0,18.0,4.0,4.0
1,ad93850f,1.0,1.0,1.0,1.0,0.0,0.0,10.0,3.214550,3.333333,20.0,7.371115,6.666667,7.0,22.0,14.0,1.0,0.0
2,0297fc1e,2.0,1.0,2.0,1.0,0.0,0.0,29.0,0.669043,1.380952,404.0,5.228295,7.214286,26.0,28.0,2.0,0.0,2.0
3,2d681dd8,1.0,1.0,1.0,1.0,0.0,0.0,1.0,NaN,1.000000,13.0,0.707107,6.500000,9.0,27.0,18.0,6.0,4.0
4,cccea85e,1.0,1.0,1.0,1.0,0.0,0.0,15.0,0.640870,1.875000,739.0,26.241971,56.846154,23.0,31.0,7.0,3.0,0.0


In [327]:
features_rance = pd.read_csv('features_numericos.csv')

In [269]:
features_rance.columns

Index(['person', 'Cantidad_visitas', 'Promedio_visitas_producto',
       'horas_mirando_productos', 'visitas_mes1', 'visitas_mes2',
       'visitas_mes3', 'visitas_mes4', 'visitas_mes5',
       'promedio_ingreso_mensual', 'registros_semana_1', 'registros_semana_2',
       'registros_semana_3', 'registros_semana_4', 'registros_semana_5',
       'promedio_registros_diarias', 'dias_ingresando',
       'promedio_visitas_hora', 'std_visitas_diarias', 'std_visitas_hora',
       'max_visitas_diarias', 'min_visitas_diarias', 'diff_min_max_diaria',
       'max_visitas_hora', 'min_visitas_hora', 'diff_min_max_hora',
       'ultimo_ingreso_dia', 'primer_ingreso_dia',
       'diferencia_primer_ultimo_dia'],
      dtype='object')

In [271]:
features_rance =features_rance[['person','visitas_mes5']]

In [223]:
['std_visitas_diarias','std_visitas_hora','Cantidad_visitas','Promedio_visitas_producto', 'visitas_mes1','visitas_mes2','visitas_mes3', 'visitas_mes4','DAY(last month 5)','DAY(first month 5)' ,'WEEKDAY(first month 5)','WEEKDAY(last month 5)','diff_std','checkout_std','visited_std']


['std_visitas_diarias',
 'std_visitas_hora',
 'Cantidad_visitas',
 'Promedio_visitas_producto',
 'visitas_mes1',
 'visitas_mes2',
 'visitas_mes3',
 'visitas_mes4',
 'DAY(last month 5)',
 'DAY(first month 5)',
 'WEEKDAY(first month 5)',
 'WEEKDAY(last month 5)',
 'diff_std',
 'checkout_std',
 'visited_std']

In [606]:
(df.loc[df['month']==5])['event'].value_counts()

viewed product       926564
brand listing        147850
visited site         143391
ad campaign hit      139793
generic listing      112294
searched products     97327
search engine hit     77717
checkout              55230
staticpage             7906
conversion             5498
lead                    350
Name: event, dtype: int64